In [63]:
import os
import open3d as o3d
import numpy as np
import torch

data_dir = "/data/lab-gatr"
disp_dir = "/data/lab-gatr/point_displacements"
patient_id = 2

In [59]:
def print_mesh(mesh):
    v = np.asarray(mesh.vertices)
    f = np.asarray(mesh.triangles)
    n = np.asarray(mesh.vertex_normals)
    print(f"Mesh: {len(v)} vertices, {len(n)} normals, {len(f)} faces")

def print_pointcloud(pcd):
    p = np.asarray(pcd.points)
    print(f"Pointcloud: {len(p)} points")

def orient_normals_outward(vertices, normals):
    center = vertices.mean(axis=0)
    print(f"Center: {center[0]:.3f}, {center[1]:.3f}, {center[2]:.3f}")
    dirs = vertices - center
    dots = np.einsum("ij,ij->i", normals, dirs)
    normals[dots < 0] *= -1
    return normals

def orient_normals_negative_zaxis(normals):
    mask = normals[:, 2] > 0
    normals[mask] *= -1
    return normals

In [74]:
patients = [os.path.join(data_dir, p) for p in os.listdir(data_dir) if p.startswith("24_")]
patient = [p for p in patients if os.path.basename(p).startswith(f"24_{patient_id:02d}")][0]
disps = torch.load(os.path.join(disp_dir, os.path.basename(patient) + ".pt"))["displacements"]

# Read not-insufflated abdominal shape
begin_mesh = o3d.io.read_triangle_mesh(os.path.join(patient, "filtered_begin_mesh.ply"))
begin_pcd = o3d.io.read_point_cloud(os.path.join(patient, "filtered_begin_pointcloud.ply"))
print_mesh(begin_mesh)
print_pointcloud(begin_pcd)

# Read insufflated abdominal shape
end_mesh = o3d.io.read_triangle_mesh(os.path.join(patient, "filtered_end_mesh.ply"))
end_pcd = o3d.io.read_point_cloud(os.path.join(patient, "filtered_end_pointcloud.ply"))
print_mesh(end_mesh)
print_pointcloud(end_pcd)

Mesh: 61097 vertices, 61097 normals, 113940 faces
Pointcloud: 61097 points
Mesh: 67844 vertices, 67844 normals, 128084 faces
Pointcloud: 67844 points


In [75]:
# new_normals = orient_normals_outward(np.asarray(begin_mesh.vertices), np.asarray(begin_mesh.vertex_normals))
new_normals = orient_normals_negative_zaxis(np.asarray(begin_mesh.vertex_normals))
begin_mesh.vertex_normals = o3d.utility.Vector3dVector(new_normals)
# o3d.io.write_triangle_mesh(os.path.join(patient, "filtered_begin_mesh_normals2.ply"), begin_mesh)

In [82]:
torch.mean(torch.norm(disps, dim=1))

tensor(0.0267)

In [30]:
pos, orientation = np.asarray(begin_mesh.vertices), np.asarray(begin_mesh.vertex_normals)